# Fall Detection RNN Model Training

## Setup

In [1]:
# Data loading
from utils import load_data

# random seed
from random import randint

# Tensorflow and Keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GRU, LSTM, Dropout, GaussianNoise, Input, Bidirectional
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow_addons.metrics import F1Score
from tensorflow.random import set_seed

# Optimization (Optuna)
import optuna
optuna.logging.set_verbosity(optuna.logging.WARNING)

# metrics / evaluation
from sklearn.metrics import f1_score
from numpy import rint

# logging
import wandb
from wandb.keras import WandbCallback

# datetime
import datetime

# timestamp for wandb
TIME = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

## Data Loading

In [2]:
X_train, X_val, X_test, y_train, y_val, y_test = load_data(win_size =200)

Samples fetched
Data fetched
all_samples shape: (194, 200, 7)
all_labels shape: (194,)
X_train shape: (135, 200, 7)
X_val shape: (30, 200, 7)
X_test shape: (29, 200, 7)
X_train shape: (135, 200, 7), 1/0 ratio: 0.4
X_val shape: (30, 200, 7), 1/0 ratio: 0.4
X_test shape: (29, 200, 7), 1/0 ratio: 0.41379310344827586


## Training

### Optimization

In [3]:
def get_model(trial, input_shape: tuple):

    dropout = trial.suggest_categorical("dropout", [0.0, 0.1, 0.2, 0.3, 0.4, 0.5])
    #recurrent_dropout = trial.suggest_categorical("recurrent_dropout", [0.0, 0.1, 0.2, 0.3, 0.4, 0.5])
    gaussian_noise = trial.suggest_float("gaussian_noise", 0.0, 0.2)
    #bidirectional = trial.suggest_categorical("bidirectional", [True, False])
    rnn_variant = trial.suggest_categorical("rnn_variant", ["gru", "lstm"])
    units = trial.suggest_categorical("n_units", [16, 32, 64, 128])
    activation = trial.suggest_categorical("activation", ["relu", "tanh", "leaky_relu"])

    config = {
        "dropout": dropout,
        #"recurrent_dropout": recurrent_dropout,
        "gaussian_noise": gaussian_noise,
        #"bidirectional": bidirectional,
        "rnn_variant": rnn_variant,
        "n_units": units,
        "activation": activation,
    }

    input = Input(shape=input_shape)
    o = GaussianNoise(gaussian_noise)(input)

    if rnn_variant == "gru":
        #if bidirectional:
        #    gru = GRU(units, return_sequences=False, activation=activation, recurrent_dropout=recurrent_dropout, dropout=dropout)
        #    o = Bidirectional(gru)(o)
        #else:
        o = GRU(units, return_sequences=True)(o)
        o = GRU(units, return_sequences=False)(o)

    elif rnn_variant == "lstm":
        #if bidirectional:
        #    lstm = LSTM(units, return_sequences=False, activation=activation, recurrent_dropout=recurrent_dropout, dropout=dropout)
        #    o = Bidirectional(lstm)(o)
        #else:
        o = LSTM(units, return_sequences=True)(o)
        o = LSTM(units, return_sequences=False)(o)

    o = Dense(units, activation=activation)(o)
    o = Dropout(dropout)(o)
    o = Dense(1, activation="sigmoid")(o)

    model = Model(inputs=input, outputs=o)
    return model, config

def objective(trial):

    learning_rate = trial.suggest_float("learning_rate", 0.001, 0.05)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128, 256])
    optimizer = trial.suggest_categorical("optimizer", ["adam", "sgd", "rmsprop"])

    model, config = get_model(trial, (X_train.shape[1], X_train.shape[2]))

    config["model_type"] = "rnn"
    config["window_size"] = 200

    config["learning_rate"] = learning_rate
    config["batch_size"] = batch_size
    config["optimizer"] = optimizer

    # init wandb
    wandb.init(entity="protechted", project=f"fall-detection", group=f"optuna_umafall_{config['window_size']}_{TIME}", reinit=True, config=config)

    if optimizer == "adam":
        optimizer = Adam(learning_rate=learning_rate)
    elif optimizer == "sgd":
        optimizer = SGD(learning_rate=learning_rate)
    elif optimizer == "rmsprop":
        optimizer = RMSprop(learning_rate=learning_rate)

    model.compile(optimizer=optimizer,
                  loss="binary_crossentropy",
                  metrics=[Precision(name="precision"), Recall(name="recall"), F1Score(name="f1", num_classes=1, threshold=0.5)])

    print(model.summary())

    early_stopping = EarlyStopping(monitor="val_f1", patience=25, restore_best_weights=True, verbose=0, mode='max')

    try:
        model.fit(X_train,
                y_train,
                batch_size=batch_size, epochs=20,
                validation_data=(X_val, y_val),
                callbacks=[early_stopping, WandbCallback(monitor="val_f1", mode="max", labels=["no fall", "fall"])])
    except Exception as e:
        print(e)
        wandb.finish()

    preds = model.predict(X_test)
    pred_labels = rint(preds)
    f1 = f1_score(y_test, pred_labels)
    return f1

In [10]:
study = optuna.create_study(
    direction="maximize",
    study_name="fall-detection-rnn",
)
study.optimize(objective, n_trials=20, timeout=600)#, n_jobs=1)

wandb.log(
            {
                "optuna_optimization_history": optuna.visualization.plot_optimization_history(
                    study
                ),
                "optuna_param_importances": optuna.visualization.plot_param_importances(
                    study
                ),
                "optuna_parallel_coordinate": optuna.visualization.plot_parallel_coordinate(
                    study
                )
            }
        )

wandb.finish()

Model: "model_11"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 200, 7)]          0         
                                                                 
 gaussian_noise_11 (Gaussian  (None, 200, 7)           0         
 Noise)                                                          
                                                                 
 bidirectional_6 (Bidirectio  (None, 256)              139264    
 nal)                                                            
                                                                 
 dropout_15 (Dropout)        (None, 256)               0         
                                                                 
 dense_22 (Dense)            (None, 128)               32896     
                                                                 
 dropout_16 (Dropout)        (None, 128)               0  

epoch,▁▂▃▃▄▅▆▆▇█
f1,▄▁▆▅▂█▆███
loss,▇▄▄█▄▂▄▁▇▁
precision,▃▁▅▄█▅▄▅▅▆
recall,▄▁▅▅▂█▆██▇
val_f1,▁▆▁▁▁▁▁▁▁█
val_loss,▁▁▁▁▁▁▁▁▁█
val_precision,▁▇▁▁▁▁▁▁▁█
val_recall,▁▄▁▁▁▁▁▁▁█
best_epoch,9
best_val_f1,0.46667


1/1 [==============================] - 0s 436ms/step


[W 2022-06-20 18:07:51,502] Trial 0 failed because of the following error: ValueError('Input y_pred contains NaN.')
Traceback (most recent call last):
  File "/Users/benschaper/Library/Caches/pypoetry/virtualenvs/ai-model-training-ZBO0jvbE-py3.8/lib/python3.8/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/c8/dsddf97j70d1c944dscbxqqw0000gn/T/ipykernel_78086/1403804534.py", line 97, in objective
    f1 = f1_score(y_test, pred_labels)
  File "/Users/benschaper/Library/Caches/pypoetry/virtualenvs/ai-model-training-ZBO0jvbE-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1132, in f1_score
    return fbeta_score(
  File "/Users/benschaper/Library/Caches/pypoetry/virtualenvs/ai-model-training-ZBO0jvbE-py3.8/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 1270, in fbeta_score
    _, _, f, _ = precision_recall_fscore_support(
  File "/Users/benschaper/Library/Caches/pypoet

ValueError: Input y_pred contains NaN.

### Individual Model

In [49]:
# load config from wandb by run name:
# activation
# "tanh"
# batch_size
# 64
# bidirectional
# true
# dropout
# 0.1
# gaussian_noise
# 0.16112220329548682
# learning_rate
# 0.026837350860689297
# model_type
# "rnn"
# n_units
# 16
# optimizer
# "sgd"
# recurrent_dropout
# 0.4
# rnn_variant
# "lstm"
# window_size
# 150

config = {
    "activation": "tanh",
    "batch_size": 64,
    "bidirectional": True,
    "dropout": 0.1,
    "gaussian_noise": 0.16112220329548682,
    "learning_rate": 0.026837350860689297,
    "model_type": "rnn",
    "n_units": 16,
    "optimizer": "sgd",
    "recurrent_dropout": 0.4,
    "rnn_variant": "lstm",
    "window_size": 150
}

In [52]:
# set seed
random_int = randint(0, 100)
set_seed(random_int)
print(f"Random seed: {random_int}")

def get_model(config, input_shape: tuple):

    dropout = config["dropout"]
    recurrent_dropout = config["recurrent_dropout"]
    gaussian_noise = config["gaussian_noise"]
    bidirectional = config["bidirectional"]
    rnn_variant = config["rnn_variant"]
    units = config["n_units"]
    activation = config["activation"]

    input = Input(shape=input_shape)
    o = GaussianNoise(gaussian_noise)(input)

    if rnn_variant == "gru":
        if bidirectional:
            gru = GRU(units, return_sequences=False, activation=activation, recurrent_dropout=recurrent_dropout, dropout=dropout)
            o = Bidirectional(gru)(o)
        else:
            o = GRU(units, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=False)(o)

    elif rnn_variant == "lstm":
        if bidirectional:
            lstm = LSTM(units, return_sequences=True, activation=activation, recurrent_dropout=recurrent_dropout, dropout=dropout)
            o = Bidirectional(lstm)(o)
            lstm = LSTM(units, return_sequences=False, activation=activation, recurrent_dropout=recurrent_dropout, dropout=dropout)
            o = Bidirectional(lstm)(o)
        else:
            o = LSTM(units, dropout=dropout, recurrent_dropout=recurrent_dropout, return_sequences=False)(o)

    o = Dense(units, activation=activation)(o)
    o = Dropout(dropout)(o)
    o = Dense(1, activation="sigmoid")(o)

    model = Model(inputs=input, outputs=o)
    return model, config



learning_rate = config["learning_rate"]
batch_size = config["batch_size"]
optimizer = config["optimizer"]

model, config = get_model(config, (X_train.shape[1], X_train.shape[2]))

# init wandb
config["seed"] = random_int
wandb.init(entity="protechted", project=f"fall-detection", group=f"optuna_{TIME}", reinit=True, config=config)

if optimizer == "adam":
    optimizer = Adam(learning_rate=learning_rate)
elif optimizer == "sgd":
    optimizer = SGD(learning_rate=learning_rate)
elif optimizer == "rmsprop":
    optimizer = RMSprop(learning_rate=learning_rate)

model.compile(optimizer=optimizer,
                loss="binary_crossentropy",
                metrics=[Precision(name="precision"), Recall(name="recall"), F1Score(name="f1", num_classes=1, threshold=0.5)])

print(model.summary())

early_stopping = EarlyStopping(monitor="val_f1", patience=50, restore_best_weights=True, verbose=0, mode='max')
model.fit(X_train,
            y_train,
            batch_size=batch_size, epochs=200,
            validation_data=(X_val, y_val),
            callbacks=[early_stopping, WandbCallback(monitor="val_f1", mode="max", labels=["no fall", "fall"])])

wandb.finish()

Random seed: 92


Model: "model_54"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_56 (InputLayer)       [(None, 150, 7)]          0         
                                                                 
 gaussian_noise_55 (Gaussian  (None, 150, 7)           0         
 Noise)                                                          
                                                                 
 bidirectional_34 (Bidirecti  (None, 150, 32)          3072      
 onal)                                                           
                                                                 
 bidirectional_35 (Bidirecti  (None, 32)               6272      
 onal)                                                           
                                                                 
 dense_108 (Dense)           (None, 16)                528       
                                                          